<a href="https://colab.research.google.com/github/AbeHandler/AbeHandler.github.io/blob/master/sa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The plan

1. Compute sa for each shard
2. Merge them w/ a mem map low memory merge
3. Shard the big SA. The sequences that are similar will go in the same shard
4. Search those shards in parallel

In [ ]:
from pydivsufsort import divsufsort, kasai

s = "aabaa"
s1 = f"{s}document9stuff"
s2 = f"{s}lorem$"
text = s1 + s2  # "aabaa document stuff$aabaa lorem"

# Compute SA and LCP
sa = divsufsort(text)
lcp = kasai(text, sa)

THRESHOLD = 5

# Display entries where LCP >= 2 and show the two common characters
print(f"{'i':>2} {'SA[i]':>5} {'SA[i-1]':>7} {'LCP[i]':>7}  {f'Common ({THRESHOLD} chars)':>20}  {'Suffix_i':>12}  {'Suffix_prev'}")
print("-" * 100)
for i in range(1, len(sa)):
    if lcp[i] >= THRESHOLD:
        pos_i = sa[i]
        pos_prev = sa[i-1]
        common = text[pos_i:pos_i + THRESHOLD]
        suffix_i = text[pos_i:]
        suffix_prev = text[pos_prev:]
        print(f"{i:2} {pos_i:5} {pos_prev:7} {lcp[i]:7}  {common!r:20}  {suffix_i!r:12}  {suffix_prev!r}")

text

 i SA[i] SA[i-1]  LCP[i]      Common (5 chars)      Suffix_i  Suffix_prev
----------------------------------------------------------------------------------------------------
 2     0      13       5  'aabaa'               'aabaadocument9stuffaabaalorem$'  '9stuffaabaalorem$'


'aabaadocument9stuffaabaalorem$'

In [ ]:
import numpy as np
import mmap

sa1 = divsufsort(s1).astype(np.uint64)
sa1.tofile("sa1.bin")

sa2 = divsufsort(s2).astype(np.uint64)
sa2.tofile("sa2.bin")


sa1_mm = np.memmap("sa1.bin", dtype=np.uint64, mode="r")
sa2_mm = np.memmap("sa2.bin", dtype=np.uint64, mode="r")

# Write the concatenated text to disk for mmap
with open("text.bin", "wb") as f:
    f.write((s1 + s2).encode())

# Memory-map the text
f_text = open("text.bin", "rb")
text_mm = mmap.mmap(f_text.fileno(), 0, access=mmap.ACCESS_READ)

# 4) Merge the two suffix arrays without loading fully into RAM
len1 = len(s1)
n1, n2 = sa1_mm.shape[0], sa2_mm.shape[0]
merged_len = n1 + n2

merged_sa = np.memmap("merged_sa.bin", dtype=np.uint64, mode="w+", shape=(merged_len,))

i = j = k = 0
while i < n1 and j < n2:
    idx1 = int(sa1_mm[i])
    idx2 = int(sa2_mm[j]) + len1
    # compare suffixes via mmap
    if text_mm[idx1:] < text_mm[idx2:]:
        merged_sa[k] = idx1
        i += 1
    else:
        merged_sa[k] = idx2
        j += 1
    k += 1

# drain remainders
while i < n1:
    merged_sa[k] = int(sa1_mm[i])
    i += 1; k += 1
while j < n2:
    merged_sa[k] = int(sa2_mm[j]) + len1
    j += 1; k += 1

merged_sa.flush()
print("Merged suffix array written to 'merged_sa.bin'")

merged_sa = np.fromfile("merged_sa.bin", dtype=np.uint64)
merged_sa_correct = divsufsort(s1 + s2).astype(np.uint64)

Merged suffix array written to 'merged_sa.bin'


In [ ]:
assert (merged_sa_correct == merged_sa).all()

In [ ]:
import numpy as np
from pathlib import Path

import shutil
from pathlib import Path

output_dir = Path("sharded_sa")

if output_dir.exists():
    shutil.rmtree(output_dir)

filepath = "merged_sa.bin"
dtype = np.uint64
N = 1  # number of shards

output_dir = Path("sharded_sa")
output_dir.mkdir(exist_ok=True)

# Open as memmap (read-only)
sa = np.memmap(filepath, dtype=dtype, mode="r")
total_len = len(sa)

# Compute shard boundaries
shard_sizes = np.array_split(np.arange(total_len), N)

# Save each shard without loading entire array
for i, idxs in enumerate(shard_sizes):
    shard = sa[idxs[0]: idxs[-1] + 1]  # slice view, not copy
    shard.tofile(output_dir / f"shard_{i:03d}.bin")


In [ ]:
def read_suffix(f, pos, max_len):
    f.seek(pos)
    return f.read(max_len)  # truncate comparison for performance

def compute_lcp_for_shard(sa, f_text, max_prefix_len=512):
    lcp = np.zeros(len(sa), dtype=np.uint32)
    for i in range(1, len(sa)):
        suffix1 = read_suffix(f_text, sa[i - 1], max_prefix_len)
        suffix2 = read_suffix(f_text, sa[i], max_prefix_len)

        # Debug: show first byte (or decode if safe)
        # print(f"{i}: {suffix1[:10]!r} vs {suffix2[:10]!r}")

        match_len = 0
        for b1, b2 in zip(suffix1, suffix2):
            if b1 != b2:
                break
            match_len += 1

        lcp[i] = match_len
    return lcp


with open("text.bin", "rb") as f_text:
    sa = np.fromfile("sharded_sa/shard_000.bin", dtype=np.uint64)

sa2 = divsufsort(text)
sa = merged_sa
with open("text.bin", "rb") as f_text:
    sa_shard = np.fromfile("sharded_sa/shard_000.bin", dtype=np.uint64).astype(np.int32)
    lcp = compute_lcp_for_shard(sa, f_text)

In [ ]:
# Display entries where LCP >= 2 and show the two common characters
print(f"{'i':>2} {'SA[i]':>5} {'SA[i-1]':>7} {'LCP[i]':>7}  {f'Common ({THRESHOLD} chars)':>20}  {'Suffix_i':>12}  {'Suffix_prev'}")
print("-" * 100)
for i in range(1, len(sa_shard)):
    if lcp[i] >= THRESHOLD:
        pos_i = sa[i]
        pos_prev = sa[i-1]
        common = text[pos_i:pos_i + THRESHOLD]
        suffix_i = text[pos_i:]
        suffix_prev = text[pos_prev:]
        print(f"{i:2} {pos_i:5} {pos_prev:7} {lcp[i]:7}  {common!r:20}  {suffix_i!r:12}  {suffix_prev!r}")

 i SA[i] SA[i-1]  LCP[i]      Common (5 chars)      Suffix_i  Suffix_prev
----------------------------------------------------------------------------------------------------
 3    19       0       5  'aabaa'               'aabaalorem$'  'aabaadocument9stuffaabaalorem$'


In [11]:
import numpy as np

string_inp = "banana$"
int_inp = np.unique(np.array(list(string_inp)), return_inverse=True)[1]
int_suffix_array = divsufsort(int_inp)
int_lcp_array = kasai(int_inp.astype(np.int32), int_suffix_array)
print(int_suffix_array, int_lcp_array)

[6 5 3 1 0 4 2] [0 1 3 0 0 2 0]


In [24]:
type(int_inp)

list

In [17]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")
enc.encode("banana$")

[3820, 2271, 3]

In [90]:
import numpy as np
import tiktoken
from pydivsufsort import divsufsort, kasai

int_inp = np.asarray([1,2,3,1, 2], dtype=np.int32)

int_suffix_array = divsufsort(int_inp)
int_lcp_array     = kasai(int_inp, int_suffix_array)

i = 0                            # this is the LCP index you were inspecting
idx1 = int_suffix_array[i]       # SA[0] == 3
idx2 = int_suffix_array[i+1]     # SA[1] == 0

print("suffix1:", int_inp[idx1:].tolist())  # [1, 2]
print("suffix2:", int_inp[idx2:].tolist())  # [1, 2, 3, 1, 2]
print("lcp[0]  :", int_lcp_array[0])        # 2


suffix1: [1, 2]
suffix2: [1, 2, 3, 1, 2]
lcp[0]  : 2


In [85]:
import numpy as np
import tiktoken
from pydivsufsort import divsufsort, kasai

int_inp = np.asarray([1,2,3,1, 2], dtype=np.int32)

int_suffix_array = divsufsort(int_inp)
int_lcp_array     = kasai(int_inp, int_suffix_array)

i = 0                            # this is the LCP index you were inspecting
idx1 = int_suffix_array[i]       # SA[0] == 3
idx2 = int_suffix_array[i+1]     # SA[1] == 0

print("suffix1:", int_inp[idx1:].tolist())  # [1, 2]
print("suffix2:", int_inp[idx2:].tolist())  # [1, 2, 3, 1, 2]
print("lcp[0]  :", int_lcp_array[0])        # 2

array([1, 2], dtype=int32)

In [91]:
from pydivsufsort import divsufsort, kasai

string_inp = "banana$"
string_suffix_array = divsufsort(string_inp)
string_lcp_array = kasai(string_inp, string_suffix_array)
print(string_suffix_array, string_lcp_array)

[6 5 3 1 0 4 2] [0 1 3 0 0 2 0]


In [108]:
from pydivsufsort import divsufsort, kasai

string_inp        = "banana$"
SA                = divsufsort(string_inp)
LCP               = kasai(string_inp, SA)

print("Suffix Array:", SA.tolist())
print("LCP Array:   ", LCP.tolist(), "\n")

for i in range(1, len(SA)):
    idx1, idx2 = SA[i-1], SA[i]
    suf1, suf2  = string_inp[idx1:], string_inp[idx2:]
    prefix = LCP[i-1]
    if prefix > 0:
        assert suf1[:prefix] == suf2[:prefix]
        print(suf1, suf2, suf1[:prefix], suf2[:prefix])


Suffix Array: [6, 5, 3, 1, 0, 4, 2]
LCP Array:    [0, 1, 3, 0, 0, 2, 0] 

a$ ana$ a a
ana$ anana$ ana ana
na$ nana$ na na


In [107]:
suf1[:prefix], suf2[:prefix]

('na', 'na')